In [1]:
import pandas as pd

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
bts = pd.read_csv('data/bus_bts.csv')
jeju_life = pd.read_csv('data/jeju_financial_life_data.csv')
weather = pd.read_csv('data/weather.csv', encoding='cp949')
rain = pd.read_csv('data/rain.csv', encoding='utf-8')

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# 내부에 결과를 출력하도록 설정
%matplotlib inline

# 시각화 한글 폰트 설정
# window 사용자
# plt.rc('font', family='Malgun Gothic')
# macOS 사용자
plt.rc('font', family='AppleGothic')

plt.rc('axes', unicode_minus=False)

# 분석에 문제가 없는 경고 메시지는 숨긴다.
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 학습 데이터와 테스트 데이터를 구분하기 위한 변수 생성
train['cue'] = 0
test['cue'] = 1

# 학습 데이터와 테스트 데이터 통합
df = pd.concat([train, test], axis=0)

In [4]:
# datetime 변수형으로 변환
df['date'] = pd.to_datetime(df['date'])

# 요일 추출 (0-월요일~6-일요일)
df['weekday'] = df['date'].dt.weekday

In [5]:
df[['weekday']].head()

,weekday
0,6
1,6
2,6
3,6
4,6


In [6]:
# 요일별 평균 탑승객 수를 구하는 함수
def week_mean():
  # 전체 데이터에서 train 데이터에 해당하는 행 추출
  train_data = df.query('cue=="0"').reset_index(drop=True)

  # 일괄적으로 1의 값을 가지는 'weekdaymean' 변수 생성
  df['weekdaymean'] = 1

  # 각 요일에 해당하는 인덱스 추출
  index0 = df.query('weekday==0').index
  index1 = df.query('weekday==1').index
  index2 = df.query('weekday==2').index
  index3 = df.query('weekday==3').index
  index4 = df.query('weekday==4').index
  index5 = df.query('weekday==5').index
  index6 = df.query('weekday==6').index

  # 인덱스를 활용해 'weekdaymean'의 값을 각 요일에 맞는 평균 탑승 탑승객 수로 변경
  df.iloc[index0, -1] = train_data.query('weekday==0')['18~20_ride'].mean()
  df.iloc[index1, -1] = train_data.query('weekday==1')['18~20_ride'].mean()
  df.iloc[index2, -1] = train_data.query('weekday==2')['18~20_ride'].mean()
  df.iloc[index3, -1] = train_data.query('weekday==3')['18~20_ride'].mean()
  df.iloc[index4, -1] = train_data.query('weekday==4')['18~20_ride'].mean()
  df.iloc[index5, -1] = train_data.query('weekday==5')['18~20_ride'].mean()
  df.iloc[index6, -1] = train_data.query('weekday==6')['18~20_ride'].mean()

  return df

# 함수 실행하여 변수 생성
df = week_mean()

In [7]:
df[['weekdaymean']].head()

,weekdaymean
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [9]:
# 날짜별 오전 시간에 탑승한 총 탑승객 수
f = df.groupby('date')['6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride', '10~11_ride'].sum().reset_index()

# 변수명 변경
f.columns = ['date', '6~7_ride_sum', '7~8_ride_sum', '8~9_ride_sum', '9~10_ride_sum', '10~11_ride_sum']

# 기존 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, f, how='left', on='date')

In [10]:
df[['date', '6~7_ride_sum', '7~8_ride_sum', '8~9_ride_sum', '9~10_ride_sum', '10~11_ride_sum']].head()

,date,6~7_ride_sum,7~8_ride_sum,8~9_ride_sum,9~10_ride_sum,10~11_ride_sum
0,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
1,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
2,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
3,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
4,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0


In [11]:
# 탑승 날짜를 datetime 형으로 변환
bts['geton_time2'] = pd.to_datetime(bts['geton_time'])

# 탑승 날짜, 탑승 정류장 코드, 탑승 노선 번호 기준으로 정렬
f = bts.sort_values(by=['geton_date', 'geton_station_code', 'bus_route_id', 'geton_time2'], ascending=True).reset_index()

In [12]:
from tqdm import notebook

# 시차를 넣기 위한 빈 리스트 생성
interval_list = []
for i in notebook.tqdm(range(0, len(f)-1)):
  # 탑승 날짜, 탑승 정류장 코드, 탑승 노선 번호가 같다면
  if ((f.iloc[i].geton_date == f.iloc[i+1].geton_date) & (f.iloc[i].geton_station_code == f.iloc[i+1].geton_station_code) & (f.iloc[i].bus_route_id == f.iloc[i+1].bus_route_id)):
    # 탑승 날짜의 차를 interval_list에 넣기
    interval_list.append(f.iloc[i+1].geton_time2 - f.iloc[i].geton_time2)

  else:
    # 같지 않다면 0을 interval_list에 넣기
    interval_list.append(0)

interval_list.insert(0, '0')

  0%|          | 0/2409413 [00:00<?, ?it/s]